자연어처리는 이번 대회를 통해 처음 접하게 되었습니다.  
우선 좋은 대회 열어주셔서 감사합니다!  
nlp에 대해 공부할 수 있는 좋은 기회였습니다.

* private score는 11등이지만 제목을 형식에 맞추지 않은 이유는 sentence-bert를 사용해서 임베딩을 했기 때문에, 이것이 pretrained model을 사용해서 cheating을 한 것이 아닌가 하는 생각이 들었기 때문입니다.  

https://medium.com/genei-technology/richer-sentence-embeddings-using-sentence-bert-part-i-ce1d9e0b1343  
처음에는 tfidf를 활용해서 과제를 진행 중이었는데, 임베딩 단계까지 pretrained model을 활용해도 된다는 공지를 보았습니다. 그래서 위의 링크와 같이 bert를 사용해서 sentence embedding을 사용해봐야 겠다는 생각을 했습니다. 하지만 현업에서 text를 다루시는 분들 사이에서도 embedding이라고 생각하는 범위가 천차만별이라 저도 어디까지 임베딩인가 고민을 많이 했습니다.

그래서 bert를 사용했기 때문에 cheating일 수 있겠다고 생각했습니다. 따라서 이번 코드공유는 앞서 시도했던 tfidf를 이용한 분류와 sentence embedding을 이용한 분류에 대한 시도로 봐주셨으면 감사하겠습니다!

In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

from scipy.sparse import hstack, vstack

from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm_notebook as tm
import re

#### tf-tdf

In [ ]:
path = 'H:/open'
train_df = pd.read_csv(path + '/data/train.csv')
test_df = pd.read_csv(path + '/data/test_x.csv')
sample_submission = pd.read_csv(path + '/data/sample_submission.csv')

# 간단한 처리만 해주었습니다.
def apply_regular_expression(text):
    text = text.lower()
    result = re.sub(' +', ' ', text)
    return result

train_df['preprocessed_text'] = train_df.text.apply(lambda x : apply_regular_expression(x))
test_df['preprocessed_text'] = test_df.text.apply(lambda x : apply_regular_expression(x))

train_text = train_df.preprocessed_text.tolist()
test_text = test_df.preprocessed_text.tolist()
label = np.asarray(train_df.author)

tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range = (1, 2),
                        max_features=250000, binary=False)

tfidf.fit(train_text)

# train / validation 따로 나누어 tfidf셋을 만들지 않았습니다.
train_tfidf = tfidf.transform(train_text).astype('float32')
test_tfidf = tfidf.transform(test_text).astype('float32')

In [3]:
def build_model():
    model = Sequential()
    model.add(Dense(128, input_dim=250000, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(5, activation='softmax'))
    return model

In [4]:
df_len = train_tfidf.shape[0]
pred = []
for i in range(10):
    
    model = build_model()
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(0.001),
                  metrics=['accuracy'])
    
    history = model.fit(x = vstack((train_tfidf[:int(df_len/10*(i))], train_tfidf[int(df_len/10*(i+1)):])),
                        y = np.concatenate([label[:int(df_len/10*(i))], label[int(df_len/10*(i+1)):]], axis = 0),
                        validation_data = (train_tfidf[int(df_len/10*i):int(df_len/10*(i+1))],
                                          label[int(df_len/10*i):int(df_len/10*(i+1))]),
                        epochs = 4)
    
    proba = model.predict_proba(test_tfidf)
    
    pred.append(proba)

Epoch 1/4
1544/1544 [==============================] - 8s 4ms/step - loss: 1.3589 - accuracy: 0.4468 - val_loss: 0.7406 - val_accuracy: 0.7724
Epoch 2/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.6826 - accuracy: 0.7718 - val_loss: 0.5723 - val_accuracy: 0.8059
Epoch 3/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.4692 - accuracy: 0.8442 - val_loss: 0.5244 - val_accuracy: 0.8167
Epoch 4/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.3507 - accuracy: 0.8846 - val_loss: 0.5124 - val_accuracy: 0.8141


c:\programdata\anaconda3\envs\torch\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/4
1544/1544 [==============================] - 6s 4ms/step - loss: 1.3404 - accuracy: 0.4628 - val_loss: 0.7374 - val_accuracy: 0.7664
Epoch 2/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.6605 - accuracy: 0.7790 - val_loss: 0.5801 - val_accuracy: 0.8005
Epoch 3/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.4453 - accuracy: 0.8542 - val_loss: 0.5388 - val_accuracy: 0.8061
Epoch 4/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.3315 - accuracy: 0.8901 - val_loss: 0.5308 - val_accuracy: 0.8099
Epoch 1/4
1544/1544 [==============================] - 6s 4ms/step - loss: 1.3476 - accuracy: 0.4542 - val_loss: 0.7277 - val_accuracy: 0.7764
Epoch 2/4
1544/1544 [==============================] - 5s 4ms/step - loss: 0.6745 - accuracy: 0.7756 - val_loss: 0.5638 - val_accuracy: 0.8038
Epoch 3/4
1544/1544 [==============================] - 6s 4ms/step - loss: 0.4586 - accuracy: 0.8506 - val_loss: 0.5195 - val_accuracy: 0.8130

In [7]:
pd.DataFrame(np.mean(pred, axis = 0))

,0,1,2,3,4
0,0.032128,0.888711,4.978818e-02,0.026257,3.115712e-03
1,0.120013,0.630619,6.400654e-02,0.042633,1.427283e-01
2,0.998843,0.000311,2.518804e-04,0.000070,5.242584e-04
3,0.009940,0.000064,9.558622e-01,0.000014,3.412066e-02
4,0.947131,0.039883,2.322925e-03,0.008892,1.770776e-03
...,...,...,...,...,...
19612,0.000040,0.999957,3.958740e-07,0.000002,1.608762e-07
19613,0.001789,0.000076,7.571139e-04,0.000061,9.973170e-01
19614,0.007273,0.991528,7.390922e-05,0.001080,4.505838e-05
19615,0.001368,0.997140,5.903693e-04,0.000745,1.570227e-04


tf-idf와 간단한 layer의 parameter 조정을 통해 다음과 같은 점수를 얻을 수 있었습니다.  
**public score : 0.2395313705

#### sentence-embedding

In [ ]:
!pip install -U sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import random
import math
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
import logging
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [12]:
print(tf.__version__)
print(torch.__version__)

2.5.0-dev20201123
1.8.0.dev20201113


In [13]:
logging.basicConfig(format = '%(asctime)s - %(message)s',
                   datefmt = "%Y-%m-%d %H-%M-%S", level = logging.INFO,
                   handlers = [LoggingHandler()])

In [4]:
path = 'H:/open/'

train_df = pd.read_csv(path+'data/train.csv')
test_df = pd.read_csv(path+'data/test_x.csv')
submission = pd.read_csv(path+'data/sample_submission.csv')

def lower_txt(text):
    result = text.lower()
    return result

train_df['lower_txt'] = train_df['text'].apply(lambda x : lower_txt(x))
test_df['lower_txt'] = test_df['text'].apply(lambda x : lower_txt(x))

train_label0_index = train_df[train_df['author'] == 0].index
train_label1_index = train_df[train_df['author'] == 1].index
train_label2_index = train_df[train_df['author'] == 2].index
train_label3_index = train_df[train_df['author'] == 3].index
train_label4_index = train_df[train_df['author'] == 4].index

train_text = train_df['lower_txt']
test_text = test_df['lower_txt']
label = train_df['author']

bert-large mean-tokens pooling model

In [15]:
model_name = ['bert-large-uncased', 'roberta-large', 'albert-large-v2', 'facebook/bart-large',
              "deepset/bert-large-uncased-whole-word-masking-squad2", "deepset/roberta-base-squad2",
              'facebook/bart-large-cnn', "facebook/bart-large-xsum"]

train_batch_size = 2

word_embedding_model = models.Transformer(model_name[0])
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                              pooling_mode_mean_tokens = True,
                              pooling_mode_cls_token = False,
                              pooling_mode_max_tokens = False)

bertl_mean_model = SentenceTransformer(modules = [word_embedding_model, pooling_model])

2020-12-07 14-29-34 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at C:\Users\mylov/.cache\torch\transformers\6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
2020-12-07 14-29-34 - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2020-12-07 14-29-36 - loading weights file https://cdn.huggingface.co/bert-large-uncased-pytorch_model.bin from cache at C:\Users\mylov/.cache\torch\transformers\73e65

bert-large max-tokens pooling model

In [16]:
train_batch_size = 2

word_embedding_model = models.Transformer(model_name[0])
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                              pooling_mode_mean_tokens = False,
                              pooling_mode_cls_token = False,
                              pooling_mode_max_tokens = True)

bertl_max_model = SentenceTransformer(modules = [word_embedding_model, pooling_model])

2020-12-07 14-30-11 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at C:\Users\mylov/.cache\torch\transformers\6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
2020-12-07 14-30-11 - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2020-12-07 14-30-11 - loading weights file https://cdn.huggingface.co/bert-large-uncased-pytorch_model.bin from cache at C:\Users\mylov/.cache\torch\transformers\73e65

In [21]:
label2int = {0 : 0, 1 : 1, 2 : 2, 3 : 3, 4 : 4}

np.random.seed(100)
train_input = []
for i in tqdm(train_df.index):
    if label[i] == 0:
        sub_text = train_text[np.random.choice(train_label0_index)]
    elif label[i] == 1:
        sub_text = train_text[np.random.choice(train_label1_index)]
    elif label[i] == 2:
        sub_text = train_text[np.random.choice(train_label2_index)]
    elif label[i] == 3:
        sub_text = train_text[np.random.choice(train_label3_index)]
    elif label[i] == 4:
        sub_text = train_text[np.random.choice(train_label4_index)]
    train_input.append(InputExample(texts = [train_text[i], sub_text], label = label2int[label[i]]))

100%|█████████████████████████████████████████████████████████████████████████| 54879/54879 [00:01<00:00, 39247.07it/s]


In [22]:
logging.info('Make Dataset')

train_dataset = SentencesDataset(train_input, model = bertl_mean_model)
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = train_batch_size)
train_loss = losses.SoftmaxLoss(model = bertl_mean_model, sentence_embedding_dimension = bertl_mean_model.get_sentence_embedding_dimension(),
                                num_labels=5)

2020-12-07 14-32-40 - Make Dataset
2020-12-07 14-32-40 - Softmax loss: #Vectors concatenated: 3


In [ ]:
num_epochs = 3
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1)
logging.info("Warmup-steps : {}".format(warmup_steps))

bertl_mean_model.fit(train_objectives = [(train_dataloader, train_loss)],
                     epochs = num_epochs,
                     warmup_steps=warmup_steps)

In [23]:
logging.info('Make Dataset')

train_dataset = SentencesDataset(train_input, model = bertl_max_model)
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = train_batch_size)
train_loss = losses.SoftmaxLoss(model = bertl_max_model, sentence_embedding_dimension = bertl_max_model.get_sentence_embedding_dimension(),
                                num_labels=5)

2020-12-07 14-37-58 - Make Dataset
2020-12-07 14-37-58 - Softmax loss: #Vectors concatenated: 3


In [ ]:
num_epochs = 3
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1)
logging.info("Warmup-steps : {}".format(warmup_steps))

bertl_max_model.fit(train_objectives = [(train_dataloader, train_loss)],
                    epochs = num_epochs,
                    warmup_steps=warmup_steps)

In [ ]:
logging.info("vector encoding")

train_mean_vec = bertl_mean_model.encode(train_text.tolist())
test_mean_vec = bertl_mean_model.encode(test_text.tolist())
train_max_vec = bertl_max_model.encode(train_text.tolist())
train_mean_vec = bertl_max_model.encode(test_text.tolist())

시간상 csv파일로 저장한 vec값들을 불러오겠습니다.

In [5]:
'''train_max_vec = np.array(pd.read_csv(path + 'data/vector/train_bertl_max3.csv'))
test_max_vec = np.array(pd.read_csv(path + 'data/vector/test_bertl_max3.csv'))
train_mean_vec = np.array(pd.read_csv(path + 'data/vector/train_bertl_mean3.csv'))
test_mean_vec = np.array(pd.read_csv(path + 'data/vector/test_bertl_mean3.csv'))'''

In [6]:
# mean, max token vector concat
train_vec = np.concatenate((train_mean_vec, train_max_vec), axis = 1)
test_vec = np.concatenate((test_mean_vec, test_max_vec), axis = 1)
label = np.array(train_df['author'])

In [9]:
def build_model():
    model = Sequential()
    model.add(Dense(128, input_dim=train_vec.shape[1], activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(5, activation='softmax'))
    return model

In [10]:
length = len(train_vec)
result = []

for i in tqdm(range(10)):
    
    dense_model = build_model()

    dense_model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(3e-5),
                  metrics=['accuracy'])

    dense_model.fit(x = np.concatenate((train_vec[:int(length/10*i)], train_vec[int(length/10*(i+1)):]), axis = 0),
                    y = np.concatenate((label[:int(length/10*i)], label[int(length/10*(i+1)):]), axis = 0),
                    validation_data = (train_vec[int(length/10*i) : int(length/10*(i+1))],
                                       label[int(length/10*i):int(length/10*(i+1))]),
                    epochs = 7)
    
    proba = dense_model.predict_proba(test_vec)
    result.append(proba)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1/7
1544/1544 [==============================] - 5s 3ms/step - loss: 0.2840 - accuracy: 0.9103 - val_loss: 0.1592 - val_accuracy: 0.9453
Epoch 2/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1611 - accuracy: 0.9481 - val_loss: 0.1572 - val_accuracy: 0.9466
Epoch 3/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1565 - accuracy: 0.9498 - val_loss: 0.1535 - val_accuracy: 0.9466
Epoch 4/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1555 - accuracy: 0.9497 - val_loss: 0.1589 - val_accuracy: 0.9471
Epoch 5/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1503 - accuracy: 0.9506 - val_loss: 0.1590 - val_accuracy: 0.9457
Epoch 6/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1444 - accuracy: 0.9547 - val_loss: 0.1532 - val_accuracy: 0.9471
Epoch 7/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1455 - accuracy: 0.9530 - val_loss: 0.1561 - val_accuracy: 0.9451

c:\programdata\anaconda3\envs\torch\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
 10%|████████▎                                                                          | 1/10 [00:27<04:04, 27.14s/it]

Epoch 1/7
1544/1544 [==============================] - 4s 2ms/step - loss: 0.2730 - accuracy: 0.9169 - val_loss: 0.1675 - val_accuracy: 0.9481
Epoch 2/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1630 - accuracy: 0.9479 - val_loss: 0.1641 - val_accuracy: 0.9486
Epoch 3/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1562 - accuracy: 0.9501 - val_loss: 0.1625 - val_accuracy: 0.9481
Epoch 4/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1515 - accuracy: 0.9521 - val_loss: 0.1640 - val_accuracy: 0.9484
Epoch 5/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1527 - accuracy: 0.9503 - val_loss: 0.1640 - val_accuracy: 0.9483
Epoch 6/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1457 - accuracy: 0.9517 - val_loss: 0.1661 - val_accuracy: 0.9475
Epoch 7/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1489 - accuracy: 0.9522 - val_loss: 0.1653 - val_accuracy: 0.9486

 20%|████████████████▌                                                                  | 2/10 [00:51<03:31, 26.43s/it]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [03:22<00:50, 25.16s/it]

Epoch 1/7
1544/1544 [==============================] - 4s 2ms/step - loss: 0.3462 - accuracy: 0.8922 - val_loss: 0.1466 - val_accuracy: 0.9541
Epoch 2/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1630 - accuracy: 0.9489 - val_loss: 0.1433 - val_accuracy: 0.9548
Epoch 3/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1592 - accuracy: 0.9489 - val_loss: 0.1433 - val_accuracy: 0.9539
Epoch 4/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1518 - accuracy: 0.9505 - val_loss: 0.1414 - val_accuracy: 0.9552
Epoch 5/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1527 - accuracy: 0.9485 - val_loss: 0.1413 - val_accuracy: 0.9550
Epoch 6/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1497 - accuracy: 0.9511 - val_loss: 0.1426 - val_accuracy: 0.9539
Epoch 7/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1503 - accuracy: 0.9505 - val_loss: 0.1408 - val_accuracy: 0.9534

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [03:47<00:25, 25.10s/it]

Epoch 1/7
1544/1544 [==============================] - 4s 2ms/step - loss: 0.2863 - accuracy: 0.9105 - val_loss: 0.1545 - val_accuracy: 0.9492
Epoch 2/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1634 - accuracy: 0.9481 - val_loss: 0.1542 - val_accuracy: 0.9484
Epoch 3/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1547 - accuracy: 0.9504 - val_loss: 0.1539 - val_accuracy: 0.9484
Epoch 4/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1520 - accuracy: 0.9508 - val_loss: 0.1502 - val_accuracy: 0.9495
Epoch 5/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1564 - accuracy: 0.9483 - val_loss: 0.1517 - val_accuracy: 0.9495
Epoch 6/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1526 - accuracy: 0.9508 - val_loss: 0.1519 - val_accuracy: 0.9490
Epoch 7/7
1544/1544 [==============================] - 3s 2ms/step - loss: 0.1486 - accuracy: 0.9512 - val_loss: 0.1552 - val_accuracy: 0.9503

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:12<00:00, 25.26s/it]


In [11]:
# 제출 파일
pred = pd.DataFrame(np.mean(result, axis = 0))
sub = pd.concat([submission.iloc[:, :1], pred], axis = 1)
print(sub)

       index         0             1             2             3         4
0          0  0.000625  1.042035e-01  8.948557e-01  2.511572e-04  0.000064
1          1  0.000104  9.998749e-01  1.341021e-06  9.505105e-06  0.000010
2          2  0.999997  1.144566e-06  1.151079e-07  3.098989e-07  0.000002
3          3  0.000014  1.350951e-03  9.986134e-01  1.225982e-05  0.000009
4          4  0.999996  8.629775e-07  1.482152e-07  2.914969e-07  0.000003
...      ...       ...           ...           ...           ...       ...
19612  19612  0.000020  9.999757e-01  7.097557e-07  1.769100e-06  0.000002
19613  19613  0.000002  2.336352e-06  3.140888e-06  6.338286e-07  0.999992
19614  19614  0.000016  9.999788e-01  7.378231e-07  1.638135e-06  0.000002
19615  19615  0.000016  9.999796e-01  6.562535e-07  1.719821e-06  0.000002
19616  19616  0.999745  6.150379e-06  1.322956e-06  3.335471e-05  0.000214

[19617 rows x 6 columns]


In [ ]:
#sub.to_csv(path + 'submission/submission.csv', index = False)

public score = 0.1745247836 / private score = 0.17445